## Compulsory Task 1

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [67]:
# Load up store_income_data.csv
import pandas as pd
import seaborn as sns
df = pd.read_csv('store_income_data_task.csv')

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [68]:
df['country']=df['country'].str.lower().str.strip() #converts string to lower case and strips whitespaces
print(df['country'])

0                united states/
1                       britain
2                 united states
3                      britain/
4                united kingdom
                 ...           
995      s. africasouth africa/
996               united states
997    united states of america
998                     england
999              united kingdom
Name: country, Length: 1000, dtype: object


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [69]:
df['country']=df['country'].str.strip("/")
df['country']=df['country'].str.strip(".")

import fuzzywuzzy
from fuzzywuzzy import process
import chardet

def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
replace_matches_in_column(df=df, column='country', string_to_match="united kingdom")

replace_matches_in_column(df=df, column='country', string_to_match="united states of america")

replace_matches_in_column(df=df, column='country', string_to_match="S. Africa")
replace_matches_in_column(df=df, column='country', string_to_match="s.a")

df.replace('britain', 'united kingdom', inplace=True)
df.replace('u.k', 'united kingdom', inplace=True)
df.replace('uk', 'united kingdom', inplace=True)
df.replace('england', 'united kingdom', inplace=True)
df.replace('britain', 'united kingdom', inplace=True)

df.replace('s. africasouth africa', 'south africa', inplace=True)
df.replace('s.a', 'south africa', inplace=True)
df.replace('sa', 'south africa', inplace=True)

df.replace('united states of america', 'united states', inplace=True)
df.replace('america', 'united states', inplace=True)

countries = df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries


There are 5 unique countries


array(['united states', 'united kingdom', 'south africa', nan, ''],
      dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column, but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [103]:
import datetime                                      
df['date_parsed'] =  pd.to_datetime(df['date_measured'], format='%d-%m-%Y') #convert column to datetime

In [104]:
df['date_today']=datetime.date.today() #get time today
df['date_today'] = pd.to_datetime(df['date_today'])

In [105]:
df['days_ago']=df['date_today']-df['date_parsed']  #subtract to work out time lapsed
df['days_ago'].dt.days #get just number of days

0       6238
1       6269
2       7114
3       6145
4      18305
       ...  
995     6358
996    11764
997     5062
998     4434
999     4112
Name: days_ago, Length: 1000, dtype: int64